In [2]:
##############   TOOK ME 10 MINUTES TO RUN THE TRAINING LOOP FOR 1 EPOCH :| FOLLOWED BY A CRASH   ##############
###   P.S. I have an AMD GPU and intel core i3 4th gen @ 1.7GHz with 4 GB RAM
###   Can't do much about it, its expensive, I'm just a student....
###   But the code is good if I could run it I would have done some more debugging.....
'''
    classifying cat or not a cat using an AUTOENCODER CNN on Tensorflow
    Dataset: Cat and Dog classes from Cifar-10 Dataset
'''
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
import cv2

In [3]:
# creating minibatches
def mini_batch(X, size):
    idx = np.random.randint(len(X), size = (size,1))
    x_bat = X[idx]
    x_bat = x_bat.reshape(size, 32, 32, 3)
    
    return x_bat

In [ ]:
# this function adds gaussian noise in the images for the purpose of training
def create_noise(image):
    noisy = np.array(tf.random_normal(shape = tf.shape(image), mean = 0, stddev = 0, dtype = tf.float32))
    return image + noisy

In [5]:
# labels
labels = pd.DataFrame(pd.read_csv('trainlabels.csv'))
# importing 5000 images of size 32x32x3
i = 0
img_base = []
for img in glob.glob("train\\*.png"):
    if labels['label'][i] == 'cat':
        img_base.append(cv2.imread(img))        
        i+=1
    else:
        i+=1
        continue
    
print('imported ', len(img_base), 'images!')

imported  5000 images!


In [6]:
# dataset preparation
x_train = np.array(img_base[0:4000]).astype(np.float32)
x_test  = np.array(img_base[4000:]).astype(np.float32)
print("training data: ", x_train.shape)
print("testing data: ", x_test.shape)

training data:  (4000, 32, 32, 3)
testing data:  (1000, 32, 32, 3)


In [7]:
# creating placeholder for input image data
inputs = tf.placeholder(tf.float32, (None, 32,32,3))
target = tf.placeholder(tf.float32, (None, 32,32,3))

# learning rate
learning_rate = 0.01

In [20]:
# encoder unit
c1 = tf.layers.conv2d(inputs, filters = 32, kernel_size = (3,3), padding = 'same', activation = tf.nn.relu)
m1 = tf.layers.max_pooling2d(c1, strides = (2,2), pool_size = (2,2), padding = "same")
# m1 shape = 16 x 16 x 32
c2 = tf.layers.conv2d(m1, filters = 32, kernel_size = (3,3), padding = 'same', activation = tf.nn.relu)
m2 = tf.layers.max_pooling2d(c2, strides = (2,2), pool_size = (2,2), padding = "same")
# m2 shape = 8 x 8 x 32
c3 = tf.layers.conv2d(m2, filters = 32, kernel_size = (3,3), padding = 'same', activation = tf.nn.relu)
encoded = tf.layers.max_pooling2d(c3, strides = (2,2), pool_size = (2,2), padding = "same")
# encoded shape = 4 x 4 x 32

# decoder unit
d1 = tf.image.resize_images(encoded, size = (8,8), method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)
c4 = tf.layers.conv2d(d1, filters = 16, kernel_size = (3,3), padding = 'same', activation = tf.nn.relu)

d2 = tf.image.resize_images(encoded, size = (16,16), method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)
c5 = tf.layers.conv2d(d2, filters = 32, kernel_size = (3,3), padding = 'same', activation = tf.nn.relu)

d3 = tf.image.resize_images(encoded, size = (32,32), method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)
c6 = tf.layers.conv2d(d3, filters = 32, kernel_size = (3,3), padding = 'same', activation = tf.nn.relu)

logits = tf.layers.conv2d(c6, filters = 3, kernel_size = (32,32), padding = 'same', activation = None)

# passing the reconstructed image in sigmoid
decoded = tf.nn.sigmoid(logits)

# sigmoid cross entropy is the loss function
cost_func = tf.nn.sigmoid_cross_entropy_with_logits(logits = logits, labels = inputs)
optimize = tf.train.AdamOptimizer(learning_rate).minimize(cost_func)

In [ ]:
#########    Run this block only if you have computation power    ##########

# creating a tensorflow session
s = tf.Session()
s.run(tf.global_variables_initializer())
# hyperparameters
epochs = 0                                 #######   Change This to any number ONLY if you have a cool NVIDIA GPU

# training loop
for i in range(epochs):
    train_batch = mini_batch(x_train, 100)
    noise_batch = np.array(create_noise(train_batch))
    s.run(optimize, {inputs : noise_batch, target : train_batch})
    
    

<class 'numpy.ndarray'>
